In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB

In [8]:
def compare(list1,list2):
    somme = 0
    for i in range(len(list1)):
        if list1[i] != list2[i]:
            somme+=1
    return somme/len(list1)

def MLE_gaussian(data):
    mu = np.mean(data)
    sigma = np.var(data)
    if sigma < 1e-6:
        sigma = 1e-6
    return mu, sigma

def MLE_laplacian(data):
    mu = np.mean(data)
    b = np.mean(np.abs(data-mu))
    if b < 1e-6:
        b = 1e-6
    return mu, b
    

def simulate_distribution(data, parameter):
    
    for distribution in list(parameter.keys()):
        if distribution == "gaussian":
            mu, sigma = parameter[distribution][0], parameter[distribution][1]
            return np.exp(-(data-mu)**2/(2 * sigma)) / np.sqrt(2*np.pi*sigma**2)

        elif distribution == "bernouilli":
            p = parameter[distribution]
            return (1-p) * data + p * (1 - data)

In [46]:
# old = np.array([1, 2, 3, 4])
# old = o
# # old = (old, [5,6,7,8])
# old.shape[0]

In [70]:
class NBC:
    """
    features_types:
            - r: real-valued
            - m: multivariate (takes value between 1 and K) ##To add support
            - b: binary (takes value in {0,1})
    """

    def __init__(self, features_types, num_classes):
        self.features_types = features_types
        self.num_classes = num_classes
        self.parameters = [[[] for j in range(len(features_types))] for i in range(num_classes)]
        self.pi = np.zeros(num_classes)
        self.dictionary_categories = None
        
    def reset(self):
        self.parameters = [[[] for j in range(len(features_types))] for i in range(num_classes)]
        self.pi = np.zeros(num_classes)
        
    def fit(self, x_train, y_train):
        x_train[:,-1] = y_train
        
#         x_y = np.concatenate((x_train, y_train.T), axis=1)
#         x_y = x_train
#         x_y[:,:-1] = y_train
        x_train = pd.DataFrame(x_train)
        

        
        for c in range(self.num_classes):
#           selct rows for y == c
            x_train_c = x_train.loc[x_train[x_train.columns[-1]] == c]
#           find out pi[c] = p(c)
            self.pi[c] = len(x_train_c) / len(y_train)
            
            category_idx = 0
            
            for j, feature in enumerate(self.features_types):
                x_train_j_c = x_train_c[x_train_c.columns[j]]

                if feature == "r":
                    ##there is two options: gaussian or laplace distribution.
                    mu_j_c, sigma_j_c = MLE_gaussian(x_train_j_c)

                    mu_j_c, b_j_c = MLE_laplacian(x_train_j_c)
                    
                    self.parameters[c][j] = {"gaussian": [mu_j_c, sigma_j_c],
                                             "laplacian": [mu_j_c, b_j_c]}
                    
                if feature == "b":
                    p_j_c = ((1-np.mean(x_train_j_c) ) * N + 1) / (N+2) ##Additive smoothing
                    self.parameters[c][j] = {"bernouilli": p_j_c}
                if feature == "m":
                    ##TODO: implement
                    print("not implemented yet")
                     
    
    
    def predict(self, x_test):
        """predicts y_test"""
        x_test = pd.DataFrame(x_test)
        #for j in range(len(x_test)):
        #    x_test[x_test.columns[j]] = x_test[x_test.columns[j]].map(self.dictionary_categories)

        best_c = -1 * np.ones(len(x_test), dtype=np.int32)
        best_log_p_y_c = - np.infty * np.ones(len(x_test))
        for c in range(self.num_classes):
            log_p_y_c = np.log(self.pi[c])
            parameters_c = self.parameters[c]
            for d in range(len(parameters_c)):
                x_test_d =  x_test[d].values
                log_p_y_c += np.log(simulate_distribution(x_test_d, parameters_c[d]))
            for i in range(len(x_test)):
                if log_p_y_c[i] > best_log_p_y_c[i]:
                    best_log_p_y_c[i] = log_p_y_c[i]
                    best_c[i] = c
        return best_c

In [71]:
from sklearn.datasets import load_iris
iris = load_iris()
X_train, y_train = iris['data'], iris['target']
# (X_train[:30])

In [72]:
y_train[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
features_types = ["r", "r", "r", "r"]
num_classes = 3

number_trials = 1000 ##we will average classification error
classification_error_NBC = [0 for i in range(10)]
classification_error_lR = [0 for i in range(10)]
classification_error_NBC_sklearn = [0 for i in range(10)]

for trial in range(number_trials):
    N, D = X_train.shape
    Ntrain = int(0.8 * N)
    shuffler = np.random.permutation(N)
    Xtrain = X_train[shuffler[:Ntrain]]
    ytrain = y_train[shuffler[:Ntrain]]
    Xtest = X_train[shuffler[Ntrain:]]
    ytest = y_train[shuffler[Ntrain:]]
    for k in range(1,11):
        lR = LogisticRegression(solver='newton-cg',C = 5, multi_class='multinomial') ##chanted lbdgs to newton-cg solver
        nbc = NBC(features_types, num_classes)
        nbc_sklearn = GaussianNB()
        
        lR.fit(Xtrain[:int(k * len(Xtrain) / 10)], ytrain[:int(k * len(Xtrain) / 10)])
        nbc.fit(Xtrain[:int(k * len(Xtrain) / 10)], ytrain[:int(k * len(Xtrain) / 10)])
        nbc_sklearn.fit(Xtrain[:int(k * len(Xtrain) / 10)], ytrain[:int(k * len(Xtrain) / 10)])
        ytest_hat_lr = lR.predict(Xtest)
        ytest_hat_nbc = nbc.predict(Xtest)
      
        ytest_hat_nbc_sklearn = nbc_sklearn.predict(Xtest)

        classification_error_lR[k-1]+=compare(ytest_hat_lr, ytest)/number_trials
        classification_error_NBC[k-1]+=compare(ytest_hat_nbc, ytest)/number_trials
        classification_error_NBC_sklearn[k-1]+=compare(ytest_hat_nbc_sklearn, ytest)/number_trials

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

...

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:72: RuntimeWarning:

In [61]:
y_train.T


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [59]:
X_train.shape

(150, 4)